In [ ]:
from datetime import datetime as dt

df_pop = pd.read_csv('/Users/spindicate/Downloads/POPTHM.csv')
df_pop.index = pd.to_datetime(df_pop.set_index('DATE').index)
df_pop = df_pop.drop('DATE', axis=1)
df_pop.index = df_pop.index.rename('Date')
df_pop = df_pop.resample('W').interpolate()
df_pop.index = df_pop.index - pd.Timedelta('1 days')
df_pop = df_pop[df_pop.index > dt(2015,1,5)]

# pivot_on = 'adj_fatalities'
# pivot_on = 'adj_fatalities'
col = 'all_ex'
pivot_on = 'adj_all_ex'

df = pd.read_excel('/Users/spindicate/Downloads/cdc-2014-2020.xlsx')
df = df[df['MMWR Year'] != 2014]
df['total'] = df.iloc[:,6:].sum(axis=1)
df = df.rename(columns={'Jurisdiction of Occurrence': 'Region'})

df.sort_values(by=['MMWR Year', 'MMWR Week']).iloc[-40:]

updatefile = '/Users/spindicate/Downloads/Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv'

df_up

df['covid'] = df[['COVID-19 (U071, Multiple Cause of Death)', 'COVID-19 (U071, Underlying Cause of Death)']].sum(axis=1)
df['all_ex'] = df['All Cause'] - df.covid
df = df[['Region', 'MMWR Year', 'MMWR Week', 'Week Ending Date', 'All Cause', 'all_ex']]
df = df.rename(columns={'Week Ending Date': 'DATE'})

rows = []
for year, df_g in df.groupby('MMWR Year'):
    for week, df_y in df_g.groupby('MMWR Week'):
        rows.append([year, week, df_y.DATE.iloc[0], df_y[col].sum()])

df_final = pd.DataFrame(rows, columns=['Year', 'Week', 'Date', 'Fatalities']).sort_values(by=['Year', 'Week'])
df_final = df_final.merge(df_pop, on='Date', how='left')
df_final['pop_growth'] = df_final.POPTHM.shift(-1) / df_final.POPTHM
df_final['pop_growth'] = df_final['pop_growth'].fillna(method='ffill')

for i, row in df_final.iterrows():
    if np.isnan(row.POPTHM):
        df_final.at[i, 'POPTHM'] = df_final.loc[i - 1, 'POPTHM'] * row.pop_growth

df_final.Year = df_final.Year.shift(periods=17)
df_final = df_final.iloc[17:-2]
weeks = df_final.Week.unique()

df_final[pivot_on] = df_final.Fatalities / df_final.POPTHM * 1000

df_final = df_final.pivot_table(index='Year', columns='Week', values=pivot_on)

df_final = df_final[weeks]
# df_final.columns = df_final.columns.droplevel()
df_final.index += 1
df_final.index = df_final.reset_index().Year.astype('int64')

df_final = df_final.cumsum(axis=1)

df_final.to_csv('/Users/spindicate/Documents/docs/covid19/charts/cdc race.csv')

df_final